In [2]:
from nltk import edit_distance
from nltk import word_tokenize
from nltk.corpus import stopwords, words
import nltk
import pandas as pd
from  spellchecker import SpellChecker
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sys import platform

In [3]:
win_path = r"C:\Users\Anton\Desktop\projects\spell_checker\csv_imports\\"
mac_linux_path = "csv_imports/"

In [4]:
if platform == "linux" or platform == "linux2":
    path = mac_linux_path
elif platform == "darwin":
    path = mac_linux_path
elif platform == "win32":
    path = win_path
else:
    path = win_path

In [5]:
porn_terms_df = pd.read_csv(path + "porn_words.csv")

In [6]:
queries_df = pd.read_csv(path + "nlp_queries.csv")
queries_df = queries_df.drop(columns=["#"])

In [7]:
queries_df["query"].unique()

array(['japanese', 'sexmex', 'hentai', ..., 'colegiala quito',
       'darla caren', 'mistress orgasm'], dtype=object)

In [8]:
queries_df["query"].fillna("", inplace=True)

In [9]:
queries_df

,id,query,cnt
0,134,japanese,60332
1,116,sexmex,55632
2,88,hentai,47325
3,360,omegle,46862
4,299,,40304
...,...,...,...
1062092,186075,scoute69stiefsohnspritztab,1
1062093,186081,dean,1
1062094,186193,colegiala quito,1
1062095,186250,darla caren,1


#### Todo
* consider plurals in some cases the same
* remove correct false-context words

Remove stopwords from queries

In [10]:
stop_words = set(stopwords.words('english')) 

In [11]:
def remove_stop_words(sentence): 
    words = sentence.split() 
    filtered_words = [word for word in words if word not in stop_words] 

    return ' '.join(filtered_words)

In [12]:
spell = SpellChecker()

#### Feed the dictionary specific porn-terms
* Pornstars names
* Studios names
* List of movies and series (Focus on games and animes)
* Akronyms

In [13]:
spell.word_frequency.load_words(['hentai', 'omegle', 'sexmex', 'roblox'])

Consider permutations

In [14]:
def spellcheck(query):
    query = remove_stop_words(query)
    query_list = word_tokenize(query)
    mispelled = spell.unknown(query_list)
    if len(mispelled) == 0:
        return query
    
    else:
        result_string = ""
        for word in mispelled:
            porn_words = porn_terms_df["term"].unique()
            print(spell.candidates(word))
            if len(list(set(spell.candidates(word)).intersection(set(porn_words)))) == 0:
                result_string += spell.correction(word) + " "
            else:
                result_string += list(set(spell.candidates(word)).intersection(set(porn_words)))[0] + " "

        return result_string

In [32]:
spellcheck("milf")

'milf'

example of queries belonging to the same cluster

In [16]:
query_1 = "japanese milf with big tit"
query_2 = "japanese milf big tits"

In [17]:
spellcheck(query_1)

'japanese milf big tit'

#### Apply to all queries

In [18]:
queries_df["shorted_corrected_query"] = queries_df["query"].apply(spellcheck)

{'pinny', 'piney', 'pina', 'pinky'}
{'dacha'}
{'compo', 'pompom', 'compos', 'momo'}
None


TypeError: 'NoneType' object is not iterable

#### trying spacy

In [1]:
import spacy
import contextualSpellCheck

nlp = spacy.load('en_core_web_sm')
contextualSpellCheck.add_to_pipe(nlp)
doc = nlp('mompov')

print(doc._.performed_spellCheck) #Should be True
print(doc._.outcome_spellCheck)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.